### Data includes engagements between counter strike players.
Data includes 10 000 clean players and 2 000 cheaters.
Data is stored as numpy arrays.

Data is in **shape (players, 30, 192, 5)**.

- 30 = number of engagements per player (always 30)
- 192 = timesteps, ticks around the engagement (5 seconds before, 1 second after)
- 5 = input variables: AttackerDeltaYaw, AttackerDeltaPitch, CrosshairToVictimYaw, CrosshairToVictimPitch, Firing

Firing is 1 or 0 and the rest are normal floats (all the data is stored in float32 numpy arrays).

In [72]:
import numpy as np

In [73]:
legit = np.load('dataset/legit.npy')
cheaters = np.load('dataset/cheaters.npy')

In [74]:
cheaters.shape, legit.shape

((2000, 30, 192, 5), (10000, 30, 192, 5))

In [75]:
cheaters = cheaters.reshape(cheaters.shape[0]*30, 192*5)
legit = legit.reshape(legit.shape[0]*30, 192*5)
legit.shape, cheaters.shape

((300000, 960), (60000, 960))

In [76]:
cheaters = np.hstack((cheaters, np.ones(cheaters.shape[0]).reshape(-1, 1)))
legit = np.hstack((legit, np.zeros(legit.shape[0]).reshape(-1, 1)))

In [77]:
np.random.shuffle(cheaters)
np.random.shuffle(legit)
data = np.concatenate((cheaters, legit), axis=0)
data.shape

(360000, 961)

In [78]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = data[:, :-1]
Y = data[:, -1]

# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [79]:
import torch
from torch import nn

print(torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32, device=device)
X_test = torch.tensor(X_test, dtype=torch.float32, device=device)
Y_train = torch.tensor(Y_train, dtype=torch.float32, device=device)
Y_test = torch.tensor(Y_test, dtype=torch.float32, device=device)

False


In [113]:
import torch.nn.functional as F

class CheaterNet(nn.Module):
    def __init__(self):
        super(CheaterNet, self).__init__()
        self.fc1 = nn.Linear(192*5, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()

    def __call__(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.sigmoid(self.fc3(x))

class CheaterCNN(nn.Module):
    def __init__(self):
        super(CheaterCNN, self).__init__()
        self.conv1 = nn.Conv1d(5, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(3840, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x.unsqueeze(1)
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten for fully connected layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [114]:
model = CheaterCNN().to(device)

from torch import optim
from tqdm import trange

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 100
batch_size = 64

for epoch in range(epochs):
    
    acc_sum = 0
    loss_sum = 0

    for i in (t := trange(0, X_train.shape[0], batch_size)):
        ins = Y_train[i:i+batch_size].reshape(-1, 1)
        optimizer.zero_grad()
        outputs = model(X_train[i:i+batch_size])
        loss = criterion(outputs, ins)
        loss.backward()
        acc = outputs.round().eq(ins).sum() / float(len(ins))

        loss_sum += loss.item()
        acc_sum += acc

        optimizer.step()
        t.set_description(f'Epoch {epoch+1}/{epochs} | Loss: {loss_sum/max(i/64, 1):.4f} | Accuracy: {acc_sum.item()/max(i/64, 1):.4f}')
        
    acc_sum = 0
    loss_sum = 0

    with torch.no_grad():
        for i in (t := trange(0, X_test.shape[0], batch_size)):
            ins = Y_test[i:i+batch_size].reshape(-1, 1)
            outputs = model(X_test[i:i+batch_size])
            loss = criterion(outputs, ins)
            acc = outputs.round().eq(ins).sum() / float(len(ins))

            loss_sum += loss.item()
            acc_sum += acc
            t.set_description(f'Test.. {epoch+1}/{epochs} | Loss: {loss_sum/max(i/64, 1):.4f} | Accuracy: {acc_sum.item()/max(i/64, 1):.4f}')

  0%|          | 0/4500 [00:00<?, ?it/s]


RuntimeError: Given groups=1, weight of size [64, 5, 3], expected input[1, 64, 960] to have 5 channels, but got 64 channels instead